In [2]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    return psnr_value, ssim_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR and SSIM
            psnr_value, ssim_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count

    return avg_psnr, avg_ssim

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}, Average PSNR: {avg_psnr:.2f}, Average SSIM: {avg_ssim:.4f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim) / 2
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\bilateral',
    'Folder2': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\blur',
    'Folder3': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\coiflet',
    'Folder4': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Haar',
    'Folder5': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\highpass',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Laplacian',
    'Folder7': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\median',
    'Folder8': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Robert',
    'Folder9': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\sobel',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\unsharpmasking',
    # Add paths for other folders here
}

# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1, Average PSNR: 25.40, Average SSIM: 0.6231
Folder: Folder2, Average PSNR: 15.67, Average SSIM: 0.2351
Folder: Folder3, Average PSNR: 14.98, Average SSIM: 0.2028
Folder: Folder4, Average PSNR: 14.98, Average SSIM: 0.2028
Folder: Folder5, Average PSNR: 11.80, Average SSIM: 0.1585
Folder: Folder6, Average PSNR: 11.36, Average SSIM: 0.1260
Folder: Folder7, Average PSNR: 15.64, Average SSIM: 0.2319
Folder: Folder8, Average PSNR: 8.55, Average SSIM: 0.0297
Folder: Folder9, Average PSNR: 7.58, Average SSIM: 0.0092

The most similar folder is: Folder1


To find the most similar picture using multiple metrics, you need to establish a scoring system that combines the individual metric values. This scoring system will help you rank images based on their overall similarity. In the modified script, I've used a simple combination of metrics:

```python
similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4
```

In this combination, I've added the averages of PSNR and SSIM and inverses of MAE and MSE. The inverses are used to ensure that higher values indicate better similarity for MAE and MSE, similar to PSNR and SSIM.

You can adjust this combination based on your preferences or domain-specific requirements. For example, you might want to give more weight to PSNR and SSIM if these metrics are more critical in your context.

Here's a breakdown of the scoring system components:

- **PSNR and SSIM:** Higher values are better, so they are added as they are.
  
- **MAE and MSE:** Lower values are better, so the inverses are used (1/x).

After calculating these scores for each image in a folder, you can then compare the similarity scores and select the image or folder with the highest score as the most similar one.

Feel free to experiment with different combinations or weights for each metric based on the specific characteristics you want to prioritize in your similarity measurement.

In [3]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\bilateral',
    'Folder2': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\blur',
    'Folder3': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\coiflet',
    'Folder4': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Haar',
    'Folder5': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\highpass',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Laplacian',
    'Folder7': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\median',
    'Folder8': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\Robert',
    'Folder9': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\sobel',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\unsharpmasking',
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 25.40
  Average SSIM: 0.6231
  Average MAE: 107.26
  Average MSE: 55.46
Folder: Folder2
  Average PSNR: 15.67
  Average SSIM: 0.2351
  Average MAE: 125.38
  Average MSE: 99.25
Folder: Folder3
  Average PSNR: 14.98
  Average SSIM: 0.2028
  Average MAE: 122.51
  Average MSE: 100.70
Folder: Folder4
  Average PSNR: 14.98
  Average SSIM: 0.2028
  Average MAE: 122.56
  Average MSE: 100.69
Folder: Folder5
  Average PSNR: 11.80
  Average SSIM: 0.1585
  Average MAE: 141.84
  Average MSE: 103.08
Folder: Folder6
  Average PSNR: 11.36
  Average SSIM: 0.1260
  Average MAE: 143.20
  Average MSE: 102.57
Folder: Folder7
  Average PSNR: 15.64
  Average SSIM: 0.2319
  Average MAE: 125.18
  Average MSE: 99.32
Folder: Folder8
  Average PSNR: 8.55
  Average SSIM: 0.0297
  Average MAE: 96.79
  Average MSE: 106.47
Folder: Folder9
  Average PSNR: 7.58
  Average SSIM: 0.0092
  Average MAE: 117.26
  Average MSE: 105.17

The most similar folder is: Folder1


Folder 1 is bilateral

In [5]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\poison',
    
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
# print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 27.22
  Average SSIM: 0.7382
  Average MAE: 111.36
  Average MSE: 46.35


in this section we have added the poisson noise 
 Poisson noise is a type of statistical noise that occurs in various imaging processes, particularly in low-light conditions, where the number of photons hitting a sensor follows a Poisson distribution.

Now, let's discuss how Poisson noise affects different domains of an image:

1. **Spatial Domain:**
   - In the spatial domain, Poisson noise manifests as random variations in pixel intensities. It appears as grainy patterns, especially in areas with low-light conditions.
   - The noise can affect the overall visual quality of the image, making it challenging to distinguish fine details, particularly in darker regions.

2. **Frequency Domain:**
   - In the frequency domain, which is often analyzed using techniques like Fourier Transform, Poisson noise introduces high-frequency components. These high-frequency components are a result of the randomness in pixel values.
   - The impact of Poisson noise in the frequency domain depends on the distribution of noise across different frequencies. If the noise is predominantly high-frequency, it can affect the fine details and edges of the image.
   - Filtering techniques, such as low-pass filters, might be applied to suppress high-frequency components and reduce the impact of Poisson noise in the frequency domain.

In general, dealing with Poisson noise involves a balance between noise reduction and preservation of image details. Various image processing techniques, such as filtering and denoising algorithms, can be applied to mitigate the effects of Poisson noise while preserving important image features. It's important to note that the choice of the appropriate technique depends on the specific characteristics of the image and the desired outcome.

در حال کلی با تغییر دادن پارامتر های مختلف می توان انواع مختلفی از این نویز را به دست اورد که برای تشخیص این حالت یک کد متفاوت زدم 
یعنی علاوه بر تغییر noise_densities = [0.5, 0.1, 1.0, 2.0]

filter_size = [5,7,3]
تغییر دادم

Smoothing filters, including median, blur, and bilateral filters, are commonly used for noise removal in images. Here's a brief explanation of how these filters work and their effects on images:

1. **Median Filter:**
   - **Operation:** The median filter replaces each pixel value with the median value of its neighboring pixels. It is particularly effective in removing impulse or salt-and-pepper noise.
   - **Effect:** Median filtering is good at preserving edges while effectively eliminating isolated noisy pixels. It works well for removing random, high-contrast noise.

2. **Blur (Gaussian) Filter:**
   - **Operation:** The blur filter, often implemented using a Gaussian kernel, calculates the weighted average of neighboring pixel values. The weights decrease with distance from the center.
   - **Effect:** Gaussian blur is effective in reducing high-frequency noise. It smoothens the image by averaging pixel values, making it useful for eliminating Gaussian noise or fine-grained noise.

3. **Bilateral Filter:**
   - **Operation:** The bilateral filter considers both spatial closeness and intensity similarity of pixels. It applies a weighted average, giving higher weights to pixels with similar intensities.
   - **Effect:** Bilateral filtering preserves edges while smoothing the image. It's effective in reducing noise while retaining fine details. This filter is particularly useful for images with textured regions and well-defined structures.

**Common Effects of Smoothing Filters:**
   - **Noise Reduction:** All three filters help in reducing different types of noise, such as Gaussian noise, salt-and-pepper noise, or speckle noise.
   - **Detail Preservation:** Median and bilateral filters are better at preserving edges and fine details compared to simple blur filters. They smooth the image while minimizing the loss of important features.
   - **Trade-off with Detail Loss:** Depending on the extent of smoothing applied, there is always a trade-off between noise reduction and loss of image details. Overly aggressive smoothing may lead to a loss of sharpness and important features.

**Considerations:**
   - The choice of a smoothing filter depends on the specific characteristics of the noise in the image and the desired outcome.
   - Parameters such as filter size and strength of smoothing should be carefully selected to balance noise reduction and detail preservation.
   - Smoothing filters are often part of more complex image processing pipelines, and their application may depend on the overall goals of the image enhancement process.

   the effect they had on the images :
   Folder: Folder bilateral
  Average PSNR: 25.40
  Average SSIM: 0.6231
  Average MAE: 107.26
  Average MSE: 55.46
Folder: Folder blur
  Average PSNR: 15.67
  Average SSIM: 0.2351
  Average MAE: 125.38
  Average MSE: 99.25
Folder: Folder median
  Average PSNR: 14.98
  Average SSIM: 0.2028
  Average MAE: 122.51
  Average MSE: 100.70

Sharpening filters enhance the contrast and edges in an image, making it appear more detailed and crisp. Here's a brief explanation of some common sharpening filters:

1. **Laplacian Filter:**
   - **Operation:** The Laplacian filter highlights regions of rapid intensity change by computing the second spatial derivative of the image. It is often used for edge detection and sharpening.
   - **Effect:** The Laplacian filter enhances high-frequency components in the image, emphasizing edges. However, it can also amplify noise.

2. **Sobel and Robert Filters:**
   - **Operation:** Sobel and Robert filters are gradient-based filters that highlight edges by computing the gradient of the image intensity.
   - **Effect:** These filters emphasize edges in different directions. Sobel filters are commonly used for both horizontal and vertical edge detection, while Robert filters are simpler and may be computationally less expensive.

3. **High-pass Filter:**
   - **Operation:** A high-pass filter allows high-frequency components (edges) to pass through while attenuating low-frequency components (smooth areas).
   - **Effect:** High-pass filters enhance edges and details in the image. They can be implemented using various techniques, including Fourier transforms.

4. **Unsharp Masking:**
   - **Operation:** Unsharp masking involves creating a blurred version of the image and subtracting it from the original to emphasize edges.
   - **Effect:** By subtracting a blurred version, unsharp masking enhances local contrast, making edges more pronounced. It can be a powerful tool for sharpening images while minimizing the risk of noise amplification.

**Common Effects of Sharpening Filters:**
   - **Edge Enhancement:** Sharpening filters enhance the contrast around edges, making them visually more prominent.
   - **Increased Image Detail:** The sharpening process often results in increased perception of fine details in the image.
   - **Noise Amplification:** Sharpening can amplify noise in the image. Care should be taken to balance the sharpening effect without introducing or enhancing noise excessively.

**Considerations:**
   - The choice of a sharpening filter depends on the specific characteristics of the image and the desired outcome.
   - Too much sharpening can lead to artifacts and unnatural-looking results. It's important to adjust parameters carefully.
   - Sharpening filters are often applied as a part of a larger image enhancement or processing pipeline, and their application may depend on the overall goals of the process.



1. **Laplacian Filter:**
   - **Operation:** The Laplacian operator is applied to an image to obtain the second spatial derivative, which highlights regions of rapid intensity changes.
   - **Mathematics:** In discrete form, the Laplacian filter is often represented as the sum of second-order differences in pixel intensities. It is mathematically expressed as ∇²f(x, y), where ∇ represents the gradient operator.
   - **Effect:** The Laplacian emphasizes edges by detecting the points where the intensity changes abruptly.

2. **Sobel and Robert Filters:**
   - **Operation:** Sobel and Robert operators compute the gradient of the image intensity in the horizontal and vertical directions, highlighting areas of intensity change.
   - **Mathematics:** The Sobel operator, for example, uses convolution with a pair of 3x3 kernels to approximate the gradient in the x and y directions. It can be represented as Gx = [-1 0 1; -2 0 2; -1 0 1] and Gy = [-1 -2 -1; 0 0 0; 1 2 1].
   - **Effect:** These filters emphasize edges by identifying the direction of the steepest intensity change.

3. **High-pass Filter:**
   - **Operation:** A high-pass filter allows high-frequency components (representing edges) to pass while attenuating low-frequency components (smooth areas).
   - **Mathematics:** In the frequency domain, high-pass filtering can be achieved by subtracting a low-pass filtered version of the image from the original. This is equivalent to performing image convolution with a high-pass kernel.
   - **Effect:** High-pass filtering enhances fine details and edges in the image.

4. **Unsharp Masking:**
   - **Operation:** Unsharp masking involves subtracting a blurred version of the image from the original to enhance local contrast.
   - **Mathematics:** Mathematically, this process is expressed as sharpened_image = original_image + k * (original_image - blurred_image), where k is a scaling factor.
   - **Effect:** Unsharp masking increases the contrast along edges, making them more visually distinct.

**Common Scientific Effects:**
   - **Frequency Response:** Sharpening filters, especially high-pass filters, can be analyzed in the frequency domain. They boost high-frequency components corresponding to edges.
   - **Convolution Operation:** Convolution is a fundamental operation in applying these filters. It involves sliding a filter kernel over the image and computing weighted sums at each position.
   - **Derivative Operators:** Sobel and Laplacian filters can be linked to derivative operators, emphasizing the importance of the rate of change of intensity.

**Considerations:**
   - **Spatial and Frequency Domain:** Sharpening filters are often analyzed both in the spatial and frequency domains to understand their impact on image features.
   - **Noise Sensitivity:** Sharpening can enhance both signal and noise. Unsharp masking, for example, can potentially amplify noise, and careful parameter tuning is required.
   - **Artifact Mitigation:** Over-sharpening can lead to artifacts such as halos around edges. Techniques like thresholding or adaptive sharpening are used to mitigate these issues.

Understanding the mathematical and scientific principles behind these filters helps in choosing the right filter for specific image processing tasks and optimizing parameters for desired results.

Folder: Folder6
  Average PSNR: 11.36
  Average SSIM: 0.1260
  Average MAE: 143.20
  Average MSE: 102.57
Folder: Folder7
  Average PSNR: 15.64
  Average SSIM: 0.2319
  Average MAE: 125.18
  Average MSE: 99.32
Folder: Folder8
  Average PSNR: 8.55
  Average SSIM: 0.0297
  Average MAE: 96.79
  Average MSE: 106.47
Folder: Folder9
  Average PSNR: 7.58
  Average SSIM: 0.0092
  Average MAE: 117.26
  Average MSE: 105.17

end of poison noise section and comparison

In [11]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\unsharp',
    'Folder2': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\boxblur',
    'Folder3': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\coiflet',
    'Folder4': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\Haar',
    'Folder5': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\highpass',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\Laplacian',
    'Folder7': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\median',
    'Folder8': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\robert',
    'Folder9': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\sobel',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\unsharp',
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 7.72
  Average SSIM: 0.0398
  Average MAE: 115.83
  Average MSE: 86.16
Folder: Folder2
  Average PSNR: 17.40
  Average SSIM: 0.2569
  Average MAE: 109.92
  Average MSE: 95.95
Folder: Folder3
  Average PSNR: 9.89
  Average SSIM: 0.0152
  Average MAE: 121.73
  Average MSE: 104.75
Folder: Folder4
  Average PSNR: 9.89
  Average SSIM: 0.0152
  Average MAE: 121.76
  Average MSE: 104.80
Folder: Folder5
  Average PSNR: 7.26
  Average SSIM: -0.0348
  Average MAE: 140.27
  Average MSE: 105.24
Folder: Folder6
  Average PSNR: 7.72
  Average SSIM: 0.0398
  Average MAE: 115.83
  Average MSE: 86.16
Folder: Folder7
  Average PSNR: 17.13
  Average SSIM: 0.3884
  Average MAE: 98.38
  Average MSE: 64.12
Folder: Folder8
  Average PSNR: 6.41
  Average SSIM: 0.0249
  Average MAE: 102.85
  Average MSE: 101.57
Folder: Folder9
  Average PSNR: 4.47
  Average SSIM: 0.0182
  Average MAE: 120.92
  Average MSE: 102.94

The most similar folder is: Folder2


for salt and peper noise it seems box blur has worked better than the others

In [12]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\saltpeper\saltpeper',
    
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
# print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 7.76
  Average SSIM: 0.0342
  Average MAE: 62.16
  Average MSE: 58.28


Salt and pepper noise, also known as impulse noise, is a type of random and irregular interference that can affect digital images. This form of noise manifests as randomly occurring bright and dark pixels, mimicking the appearance of grains of salt and pepper scattered across the image. Here's a detailed explanation of how salt and pepper noise affects pictures:

1. **Introduction to Salt and Pepper Noise:**
   - **Random Intensity Changes:** Salt and pepper noise introduces random, sudden intensity changes in an image. Some pixels become very bright (salt), while others become very dark (pepper), disrupting the overall visual appearance.

2. **Appearance in Images:**
   - **Isolated Pixels:** Salt and pepper noise typically appears as isolated pixels with extreme intensity values. These isolated pixels stand out from the surrounding regions and can significantly affect the visual quality of the image.

3. **Impact on Image Quality:**
   - **Degrades Image Quality:** The presence of salt and pepper noise degrades the overall quality of an image, making it visually unappealing. The noise disrupts the smoothness of the image and makes it challenging to discern details.

4. **Influence on Image Analysis:**
   - **Misleading Analysis:** In image analysis and processing tasks, salt and pepper noise can mislead algorithms and affect the accuracy of measurements or feature extraction. The presence of randomly intense pixels can lead to erroneous conclusions in image-based applications.

5. **Effects on Image Processing:**
   - **Filtering Challenges:** Traditional image processing techniques, such as filtering and thresholding, may face challenges when dealing with salt and pepper noise. The isolated nature of the noise makes it difficult to address using standard smoothing or blurring methods.

6. **Causes and Occurrence:**
   - **Sensor or Transmission Issues:** Salt and pepper noise can originate from various sources, including errors during image acquisition, transmission, or storage. Malfunctioning sensors, communication errors, or data corruption can introduce this type of noise.

7. **Mitigation Strategies:**
   - **Median Filtering:** Median filtering is a common technique for mitigating salt and pepper noise. This filter replaces the pixel value with the median value of neighboring pixels, effectively removing isolated noisy pixels while preserving edges.
   - **Adaptive Filtering:** Adaptive filters adjust their parameters based on local image characteristics. These filters can be effective in reducing noise while preserving image details.

8. **Preventive Measures:**
   - **Signal-to-Noise Ratio (SNR):** Improving the signal-to-noise ratio during image acquisition and transmission can help minimize the impact of salt and pepper noise. High-quality sensors and robust communication protocols contribute to better SNR.

In summary, salt and pepper noise detrimentally affects images by introducing random, isolated pixel intensities. It degrades image quality, influences image analysis tasks, and poses challenges for traditional filtering methods. Understanding the characteristics of this noise is crucial for selecting appropriate mitigation strategies in image processing applications.

Noise removal filters are essential in image processing to mitigate the impact of various types of noise, including salt and pepper noise. Different filters employ distinct techniques to address noise, and their effectiveness depends on the characteristics of the noise and the desired outcome. Let's explore how various noise removal filters can help fix salt and pepper noise:

### Smoothing Filters:
1. **Median Filter:**
   - **Operation:** Replaces each pixel value with the median value of its neighboring pixels.
   - **Effect on Salt and Pepper Noise:** Effective for removing isolated bright and dark pixels, as the median value is less affected by extreme values.

2. **Blur (Gaussian) Filter:**
   - **Operation:** Computes the weighted average of neighboring pixel values using a Gaussian kernel.
   - **Effect on Salt and Pepper Noise:** Smoothens the image by reducing high-frequency components, helping to eliminate isolated noisy pixels.

3. **Bilateral Filter:**
   - **Operation:** Considers both spatial closeness and intensity similarity of pixels in its weighted averaging.
   - **Effect on Salt and Pepper Noise:** Preserves edges while reducing noise, making it suitable for images with both textured regions and well-defined structures.

### Sharpening Filters:
1. **Laplacian Filter:**
   - **Operation:** Emphasizes regions of rapid intensity change by computing the second spatial derivative.
   - **Effect on Salt and Pepper Noise:** Can enhance edges and features, but it may also amplify noise. Usually applied after noise reduction.

2. **Sobel and Robert Filters:**
   - **Operation:** Gradient-based filters that highlight edges by computing the gradient of the image intensity.
   - **Effect on Salt and Pepper Noise:** Emphasizes edges in different directions, aiding in edge detection after noise reduction.

3. **High-pass Filter:**
   - **Operation:** Allows high-frequency components (edges) to pass while attenuating low-frequency components.
   - **Effect on Salt and Pepper Noise:** Enhances edges and fine details, but careful adjustment is needed to avoid amplifying noise.

4. **Unsharp Masking:**
   - **Operation:** Enhances local contrast by subtracting a blurred version from the original.
   - **Effect on Salt and Pepper Noise:** Helps recover details and edges while reducing the impact of noise. Often used in conjunction with noise reduction.

### Wavelet Filters:
1. **Haar Wavelet:**
   - **Operation:** Decomposes the image into approximation and detail coefficients at different scales.
   - **Effect on Salt and Pepper Noise:** Useful for denoising as it separates image details, allowing noise removal at various scales.

2. **Coiflet Wavelet:**
   - **Operation:** Similar to the Haar wavelet but with a more complex wavelet function.
   - **Effect on Salt and Pepper Noise:** Can effectively remove noise while preserving important features in the image.

### General Considerations:
- **Filter Parameters:** The effectiveness of these filters often depends on parameters such as filter size, strength of smoothing or sharpening, and threshold values.
  
- **Trade-off:** There is a trade-off between noise reduction and detail preservation. Aggressive noise reduction may result in loss of image details.

- **Combination:** In practice, a combination of filters or a multi-stage approach may be employed for optimal noise removal while preserving image features.

Choosing the appropriate filter or combination of filters depends on the specific characteristics of the salt and pepper noise in the image and the desired outcome. Experimentation and careful parameter tuning are often necessary to achieve the best results.

In [13]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\unsharp',
    'Folder2': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\boxblur',
    'Folder3': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\coiflet',
    'Folder4': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\Haar',
    'Folder5': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\highpass',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\Laplacian',
    'Folder7': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\median',
    'Folder8': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\robert',
    'Folder9': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\sobel',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\speckle\unsharp',
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 9.95
  Average SSIM: 0.0325
  Average MAE: 111.83
  Average MSE: 105.47
Folder: Folder2
  Average PSNR: 12.42
  Average SSIM: 0.1894
  Average MAE: 113.26
  Average MSE: 102.74
Folder: Folder3
  Average PSNR: 12.91
  Average SSIM: 0.0664
  Average MAE: 106.20
  Average MSE: 105.00
Folder: Folder4
  Average PSNR: 12.91
  Average SSIM: 0.0664
  Average MAE: 106.24
  Average MSE: 105.00
Folder: Folder5
  Average PSNR: 8.77
  Average SSIM: 0.0091
  Average MAE: 126.04
  Average MSE: 105.39
Folder: Folder6
  Average PSNR: 9.95
  Average SSIM: 0.0325
  Average MAE: 111.83
  Average MSE: 105.47
Folder: Folder7
  Average PSNR: 12.05
  Average SSIM: 0.1691
  Average MAE: 110.98
  Average MSE: 102.98
Folder: Folder8
  Average PSNR: 6.47
  Average SSIM: 0.0416
  Average MAE: 100.87
  Average MSE: 102.66
Folder: Folder9
  Average PSNR: 7.33
  Average SSIM: 0.0265
  Average MAE: 135.55
  Average MSE: 104.12

The most similar folder is: Folder4


it seems for speckle noise Haar had better voice removal 

In [3]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_metrics(original_image, compared_image):
    # Resize images to a common size
    common_size = (min(original_image.shape[0], compared_image.shape[0]),
                   min(original_image.shape[1], compared_image.shape[1]))
    
    original_image_resized = cv2.resize(original_image, common_size)
    compared_image_resized = cv2.resize(compared_image, common_size)

    original_gray = cv2.cvtColor(original_image_resized, cv2.COLOR_BGR2GRAY)
    compared_gray = cv2.cvtColor(compared_image_resized, cv2.COLOR_BGR2GRAY)

    psnr_value = psnr(original_gray, compared_gray, data_range=original_gray.max() - original_gray.min())
    ssim_value, _ = ssim(original_gray, compared_gray, full=True)

    # Calculate MAE and MSE
    mae_value = np.mean(np.abs(original_gray - compared_gray))
    mse_value = np.mean((original_gray - compared_gray)**2)

    return psnr_value, ssim_value, mae_value, mse_value

def compare_with_folder(original_image, folder_path):
    avg_psnr = 0
    avg_ssim = 0
    avg_mae = 0
    avg_mse = 0
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            compared_image = cv2.imread(os.path.join(folder_path, filename))

            # Calculate PSNR, SSIM, MAE, and MSE
            psnr_value, ssim_value, mae_value, mse_value = calculate_metrics(original_image, compared_image)

            # Accumulate values for averaging
            avg_psnr += psnr_value
            avg_ssim += ssim_value
            avg_mae += mae_value
            avg_mse += mse_value
            count += 1

    # Average the values
    avg_psnr /= count
    avg_ssim /= count
    avg_mae /= count
    avg_mse /= count

    return avg_psnr, avg_ssim, avg_mae, avg_mse

def find_most_similar_folder(original_image, folders):
    max_similarity = -1
    most_similar_folder = None

    for folder_name, folder_path in folders.items():
        avg_psnr, avg_ssim, avg_mae, avg_mse = compare_with_folder(original_image, folder_path)
        print(f"Folder: {folder_name}")
        print(f"  Average PSNR: {avg_psnr:.2f}")
        print(f"  Average SSIM: {avg_ssim:.4f}")
        print(f"  Average MAE: {avg_mae:.2f}")
        print(f"  Average MSE: {avg_mse:.2f}")

        # Update the most similar folder
        similarity_score = (avg_psnr + avg_ssim + (1/avg_mae) + (1/avg_mse)) / 4  # Combine all metrics
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_folder = folder_name

    return most_similar_folder

# Path to the directory containing the main image
main_image_path = r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\1.png'
main_image = cv2.imread(main_image_path)

# Paths to the 11 folders containing images for comparison
folders = {
    'Folder1': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\unsharp',
    'Folder2': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\boxblur',
    'Folder3': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\coiflet',
    'Folder4': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\Haar',
    'Folder5': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\highpass',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\Laplacian',
    'Folder7': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\median',
    'Folder8': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\robert',
    'Folder9': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\sobel',
    'Folder6': r'C:\Users\Lenovo Laptop\Downloads\DS\poison\document\guasian\unsharp',
    # Add paths for other folders here
}
# Find the most similar folder
most_similar_folder = find_most_similar_folder(main_image, folders)

# Print the result
print(f"\nThe most similar folder is: {most_similar_folder}")


Folder: Folder1
  Average PSNR: 11.36
  Average SSIM: 0.0866
  Average MAE: 118.31
  Average MSE: 105.34
Folder: Folder2
  Average PSNR: 12.62
  Average SSIM: 0.2230
  Average MAE: 117.94
  Average MSE: 103.93
Folder: Folder3
  Average PSNR: 14.11
  Average SSIM: 0.1045
  Average MAE: 132.29
  Average MSE: 102.67
Folder: Folder4
  Average PSNR: 14.11
  Average SSIM: 0.1045
  Average MAE: 132.29
  Average MSE: 102.67
Folder: Folder5
  Average PSNR: 7.68
  Average SSIM: 0.0053
  Average MAE: 123.13
  Average MSE: 104.57
Folder: Folder6
  Average PSNR: 11.36
  Average SSIM: 0.0866
  Average MAE: 118.31
  Average MSE: 105.34
Folder: Folder7
  Average PSNR: 12.38
  Average SSIM: 0.2076
  Average MAE: 118.67
  Average MSE: 103.47
Folder: Folder8
  Average PSNR: 6.48
  Average SSIM: 0.0411
  Average MAE: 100.82
  Average MSE: 102.77
Folder: Folder9
  Average PSNR: 9.84
  Average SSIM: 0.0305
  Average MAE: 127.04
  Average MSE: 105.23

The most similar folder is: Folder3


for guasian noise coiflet was better